In [1]:
import re
def logPreprocess(logfile):
    with open( logfile,'r',encoding='utf-8',errors='ignore') as f:
        data = f.read()
    linedata=[]
    dcount=0
    RawData = []
    c =0
    g = 0
    l = 0

    Special_char_pat = re.compile('[\=\-\#{%}]{2,}')
    datepat = re.compile('\d{2}-\d{2}\s\d{2}:\d{2}:\d{2}\.\d{2,}\s*\d{2,}\s*\d{2,}\s*\w\s\S*\s*:\s*(?:java\..*)')
    gpatt = re.compile('\d{2}-\d{2}\s*\d{2}:\d{2}:\d{2}\.\d{2,}\s*\d{2,}\s*\d{2,}\s*\w\s\S*\s*:\s*(at)')
    javagrp = re.compile('\d{2}-\d{2}\s*\d{2}:\d{2}:\d{2}\.\d{2,}\s*\d{2,}\s*\d{2,}\s*\w\s\S*\s*:\s*(?:java\.)')
    allpat = re.compile('\d{2}-\d{2}\s\d{2}:\d{2}:\d{2}\.\d{2,}\s*\d{2,}\s*\d{2,}\s*\w\s\S*\s*:')
    Processgrp = re.compile('\d{2}-\d{2}\s*\d{2}:\d{2}:\d{2}\.\d{2,}\s*\d{2,}\s*\d{2,}\s*\w\s\S*\s*:\s*(Process\:)')
    
    #Process:
    data = re.sub(gpatt, ' at', data)
    data = re.sub(javagrp, ' java.', data)
    content = re.sub(Processgrp, ' Process:', data)
    #content = re.sub('\n', ' ', data)
    # date_time_pattern = r"\[\d{4}-\d{2}-\d{2}\s\d{2}:\d{2}:\d{2},\d{2,}\]"

    # Find all matches of date time pattern
    matches = re.findall(allpat, content)

    # Split the content using date time pattern
    split_content = re.split(allpat, re.sub('\n',' ',content))
    #print("Before split_content : ",split_content)
    # Remove the first empty string from split_content
    split_content = split_content[1:]
    #print("After split_content : ",split_content)

    # Combine matches and split_content
    combined_content = [m + s for m, s in zip(matches, split_content)]
    return combined_content
#logfile = "C:\\HCL\\iLA\\Versions\\Cisco_PSI_Versions\\SampleLogs\\sdtvTest.txt"
logfile = "C:\HCL\iLA\Versions\Cisco_PSI_Versions\SampleLogs\SonyDTV_\START_u0.txt"
lines = logPreprocess(logfile)
print(len(lines))
print(lines)

13
['02-21 00:36:41.324  3030  3061 D CompatibilityChangeReporter: Compat change id reported: 135634846; UID 10113; state: DISABLED ', '02-21 00:36:43.324  3030  3061 D CompatibilityChangeReporter: Compat change id reported: 177438394; UID 10113; state: DISABLED ', '02-21 00:37:45.324  3030  3061 D CompatibilityChangeReporter: Compat change id reported: 135772972; UID 10113; state: DISABLED ', '02-21 00:37:47.324  3030  3061 D CompatibilityChangeReporter: Compat change id reported: 135754954; UID 10113; state: ENABLED ', '02-21 00:37:56.666  3030  3883 I ActivityTaskManager: START u0 {act=android.intent.action.MAIN cat=[android.intent.category.LEANBACK_LAUNCHER] flg=0x10000000 pkg=com.sony.dtv.tvx cmp=com.sony.dtv.tvx/.MainActivity} from uid 10128 ', '02-21 00:38:41.325  3030  3139 D CompatibilityChangeReporter: Compat change id reported: 143937733; UID 10113; state: ENABLED ', '02-21 00:38:41.332  3030  4974 D CompatibilityChangeReporter: Compat change id reported: 135634846; UID 1005

In [18]:
from datetime import datetime, timedelta
import pandas as pd
import re

def filterLogs(ito_date, ito_time,ifrom_date,ifrom_time,lines):
    print("Filter Log Files...")

    # Replace this line with the path to your log file
    #log_file_path = "C:\\HCL\\iLA\\Versions\\Cisco_PSI_Versions\\SampleLogs\\sdtvTest.txt"
    if ito_date and ito_time and ifrom_date and ifrom_time:
        print("Got both inputs from and to datetime")
        ito_date_time = f"{ito_date} {ito_time}"
        ifrom_date_time = f"{ifrom_date} {ifrom_time}"

        # Convert the input date and time to a datetime object
        from_date = datetime.strptime(ifrom_date_time, "%m-%d %H:%M:%S.%f")
        to_date = datetime.strptime(ito_date_time, "%m-%d %H:%M:%S.%f")
        print("from_date:", from_date)
        print("to_date:", to_date)
        date_df = pd.DataFrame(lines, columns=['Content'])

        # Extract and convert timestamps to datetime objects
        date_df['DateTime'] = date_df['Content'].str.extract(r'(\d{2}-\d{2}\s\d{2}:\d{2}:\d{2}\.\d{3})')
        date_df['DateTime'] = pd.to_datetime(date_df['DateTime'], format="%m-%d %H:%M:%S.%f")

        # Filter log entries within the specified time range
        mask = (date_df['DateTime'] >= from_date) & (date_df['DateTime'] <= to_date)
        filtered_df = date_df[mask]

        # Print the filtered log entries
        print("Filtered Log Entries:")
        #print(filtered_df)
        return filtered_df,date_df
    elif ito_date and ito_time:
        
        # Combine the provided date and time into a single string
        input_date_time = f"{ito_date} {ito_time}"
        print("input_date_time:", input_date_time)

        # Convert the input date and time to a datetime object
        input_datetime = datetime.strptime(input_date_time, "%m-%d %H:%M:%S.%f")

        # Calculate the time range
        from_date = input_datetime - timedelta(minutes=1)
        to_date = input_datetime + timedelta(minutes=1)
        print("from_date:", from_date)
        print("to_date:", to_date)
        # Read the log file
    #     with open(log_file_path, 'r') as f:
    #         lines = f.readlines()

        # Create a DataFrame from the log content
        date_df = pd.DataFrame(lines, columns=['Content'])

        # Extract and convert timestamps to datetime objects
        date_df['DateTime'] = date_df['Content'].str.extract(r'(\d{2}-\d{2}\s\d{2}:\d{2}:\d{2}\.\d{3})')
        date_df['DateTime'] = pd.to_datetime(date_df['DateTime'], format="%m-%d %H:%M:%S.%f")

        # Filter log entries within the specified time range
        mask = (date_df['DateTime'] >= from_date) & (date_df['DateTime'] <= to_date)
        filtered_df = date_df[mask]

        # Print the filtered log entries
        print("Filtered Log Entries:")
        #print(filtered_df)
        return filtered_df,date_df
    elif ito_time and ifrom_time:
        print("only from and to times are provided")
        input_date_time = ito_time
        # Convert the input time to a datetime object
        ito_time = datetime.strptime(ito_time, "%H:%M:%S.%f")
        ifrom_time = datetime.strptime(ifrom_time, "%H:%M:%S.%f")

        # Calculate the time range (1 minute before and after the input time)
        from_time = ifrom_time
        to_time = ito_time
        print("from_time:", from_time)
        print("to_time:", to_time)

        # Create a DataFrame from the log content
        time_df = pd.DataFrame(lines, columns=['Content'])

        # Extract and convert timestamps to datetime objects
        time_df['Time'] = time_df['Content'].str.extract(r'(\d{2}:\d{2}:\d{2}\.\d{3})')
        time_df['Time'] = pd.to_datetime(time_df['Time'], format="%H:%M:%S.%f")

        # Filter log entries within the specified time range
        mask = (time_df['Time'] >= from_time) & (time_df['Time'] <= to_time)
        filtered_df = time_df[mask]

        # Print the filtered log entries
        print("Filtered Log Entries:")
        return filtered_df, time_df
    elif not ito_date and ito_time:
        input_date_time = ito_time
        # Convert the input time to a datetime object
        input_time = datetime.strptime(input_date_time, "%H:%M:%S.%f")

        # Calculate the time range (1 minute before and after the input time)
        from_time = input_time - timedelta(minutes=1)
        to_time = input_time + timedelta(minutes=1)
        print("from_time:", from_time)
        print("to_time:", to_time)

        # Create a DataFrame from the log content
        time_df = pd.DataFrame(lines, columns=['Content'])

        # Extract and convert timestamps to datetime objects
        time_df['Time'] = time_df['Content'].str.extract(r'(\d{2}:\d{2}:\d{2}\.\d{3})')
        time_df['Time'] = pd.to_datetime(time_df['Time'], format="%H:%M:%S.%f")

        # Filter log entries within the specified time range
        mask = (time_df['Time'] >= from_time) & (time_df['Time'] <= to_time)
        filtered_df = time_df[mask]

        # Print the filtered log entries
        print("Filtered Log Entries:")
        return filtered_df, time_df

In [ ]:
#Test 1 :  ito_date, ito_time,ifrom_date,ifrom_time,lines
# from_date = "02-21"
# to_date = "02-21"
# #to_date = "02-21"
# from_time = "00:37:41.336"
# to_time = "00:40:41.336"
# #to_time = "00:38:41.336"
# df,date_df = filterLogs(to_date, to_time,from_date,from_time,lines)
# df.head(11)

#Test 2 : 
from_date = "02-21"
to_date = "02-21"
#to_date = "02-21"
from_time = "00:37:41.336"
to_time = "00:40:41.336"
#to_time = "00:38:41.336"
df,date_df = filterLogs(to_date, to_time,from_date,from_time,lines)
df.head(11)

In [3]:
#df.loc[df["Content"].str.contains(": START u0")]

In [4]:
if df["Content"].str.contains("FATAL EXCEPTION:").any():
    print("yes, Found Fatal exception in the given time")
    #print(df.loc[df["Content"].str.contains("FATAL EXCEPTION:")])
    fdf = df.loc[df["Content"].str.contains("FATAL EXCEPTION:")]
    pat = r"Process:\s(.*\..*\..*\..*)\,\sPID:\s(\d{4})"
    print("fdf : \n",fdf)
    Package_Name = re.search(pat,fdf.iloc[0,0]).group(1)
    PID = re.search(pat,fdf.iloc[0,0]).group(2)
    print("Package_Name : ",Package_Name)
    
    print("PID : ",PID)
elif date_df["Content"].str.contains(": START u0").any():
    print("No Exceptions found in the given time, getting the application name")
    print(date_df.loc[date_df["Content"].str.contains(": START u0")])
    start_df = date_df.loc[date_df["Content"].str.contains(": START u0")]
    start_u0_lines = list(start_df["Content"])
    #print(list(start_df["Content"]))
    print(start_u0_lines)
    last_start_u0_line = start_u0_lines[-1]
    print("last_start_u0_line :\n",last_start_u0_line)
    spat = r"pkg\=(.*\..*\..*\..*)\scmp"
    last_package_name = re.search(spat,last_start_u0_line).group(1)
    print("last_application_name : ",last_package_name)
    
    
else:
    print("Not found")


No Exceptions found in the given time, getting the application name
                                              Content                DateTime
4   02-21 00:37:56.666  3030  3883 I ActivityTaskM... 1900-02-21 00:37:56.666
9   02-21 00:38:41.324  3030  3517 I ActivityTaskM... 1900-02-21 00:38:41.324
12  02-21 00:38:41.324  3030  3517 I ActivityTaskM... 1900-02-21 00:38:41.324
['02-21 00:37:56.666  3030  3883 I ActivityTaskManager: START u0 {act=android.intent.action.MAIN cat=[android.intent.category.LEANBACK_LAUNCHER] flg=0x10000000 pkg=com.sony.dtv.tvx cmp=com.sony.dtv.tvx/.MainActivity} from uid 10128 ', '02-21 00:38:41.324  3030  3517 I ActivityTaskManager: START u0 {act=android.intent.action.MAIN cat=[android.intent.category.LEANBACK_LAUNCHER] flg=0x10000000 pkg=com.sony.dtv.recap cmp=com.sony.dtv.tvx/.MainActivity} from uid 10128 ', '02-21 00:38:41.324  3030  3517 I ActivityTaskManager: START u0 {act=android.intent.action.MAIN cat=[android.intent.category.LEANBACK_LAUNCHER] flg=0

In [ ]:
inputApplication ="Application_Name"
inputTime = 

In [10]:
from datetime import datetime, timedelta
import pandas as pd

def filterLogs(date, time, lines):
    print("Filter Log Files...")

    # Check if only time is provided, and if so, use the current date
    if not date and time:
        #date = datetime.now().strftime("%m-%d")

        # Combine the provided date and time into a single string
        input_time = time
        print("input_time:", input_time)

        # Convert the input date and time to a datetime object
        input_time = datetime.strptime(input_time, "%H:%M:%S.%f")

        # Calculate the time range
        from_date = input_time - timedelta(minutes=1)
        to_date = input_time + timedelta(minutes=1)
        print("from_date:", from_date)
        print("to_date:", to_date)

        # Create a DataFrame from the log content
        date_df = pd.DataFrame(lines, columns=['Content'])

        # Extract and convert timestamps to datetime objects
        date_df['Time'] = date_df['Content'].str.extract(r'(\d{2}:\d{2}:\d{2}\.\d{3})')
        date_df['Time'] = pd.to_datetime(date_df['Time'], format="%H:%M:%S.%f")

        # Filter log entries within the specified time range
        mask = (date_df['Time'] >= from_date) & (date_df['Time'] <= to_date)
        filtered_df = date_df[mask]

        # Print the filtered log entries
        print("Filtered Log Entries:")
        return filtered_df, date_df

# # Example with date and time
# date = "02-21"
# time = "00:38:41.336"
# lines = []  # Replace with your log lines
# df, date_df = filterLogs(date, time, lines)
# print(df.head(10))

# Example with only time (ignores date)
date = ""  # An empty string indicates no date
time = "00:38:41.336"
lines = []  # Replace with your log lines
df, date_df = filterLogs(date, time, lines)
print(df.head(10))


Filter Log Files...
input_time: 00:38:41.336
from_date: 1900-01-01 00:37:41.336000
to_date: 1900-01-01 00:39:41.336000
Filtered Log Entries:
Empty DataFrame
Columns: [Content, Time]
Index: []


# get application name from exception

In [3]:
import re

#txt = "Process: com.sony.dtv.iswpowersavingsettings, PID: 7378"
txt = "02-21 00:38:41.336  7378  7378 E AndroidRuntime: FATAL EXCEPTION: main  Process: com.sony.dtv.Quicksettings, PID: 7378  java.lang.IllegalStateException: IPowerSetting is not ready yet.  at android.os.Parcel.createExceptionOrNull(Parcel.java:2434)  at android.os.Parcel.createException(Parcel.java:2410) "
pat = r"Process:\s(.*\..*\..*\..*)\,\sPID:\s(\d{4})"
Package_Name = re.search(pat,txt).group(1)
PID = re.search(pat,txt).group(2)
print("Package_Name : ",Package_Name)
Application_Name = Package_Name.split(".")[-1]
print("Application_Name : ",Application_Name)
print("PID : ",PID)

Package_Name :  com.sony.dtv.Quicksettings
Application_Name :  Quicksettings
PID :  7378


# get last application name from START u0 keyword

In [49]:
stxt = "02-21 00:38:41.324  3030  3517 I ActivityTaskManager: START u0 {act=android.intent.action.MAIN cat=[android.intent.category.LEANBACK_LAUNCHER] flg=0x10000000 pkg=com.sony.dtv.tvx cmp=com.sony.dtv.tvx/.MainActivity} from uid 10128"
#pkg=com.sony.dtv.tvx 
spat = r"pkg\=(.*\..*\..*\..*)\scmp"

re.search(spat,stxt).group(1)


'com.sony.dtv.tvx'

In [ ]:
from datetime import datetime, timedelta
import pandas as pd
import re

def filterLogs(date, time,lines):
    print("Filter Log Files...")

    # Replace this line with the path to your log file
    #log_file_path = "C:\\HCL\\iLA\\Versions\\Cisco_PSI_Versions\\SampleLogs\\sdtvTest.txt"

    # Combine the provided date and time into a single string
    input_date_time = f"{date} {time}"
    print("input_date_time:", input_date_time)

    # Convert the input date and time to a datetime object
    input_datetime = datetime.strptime(input_date_time, "%m-%d %H:%M:%S.%f")

    # Calculate the time range
    from_date = input_datetime - timedelta(minutes=1)
    to_date = input_datetime + timedelta(minutes=1)
    print("from_date:", from_date)
    print("to_date:", to_date)
    # Read the log file
#     with open(log_file_path, 'r') as f:
#         lines = f.readlines()

    # Create a DataFrame from the log content
    date_df = pd.DataFrame(lines, columns=['Content'])

    # Extract and convert timestamps to datetime objects
    date_df['DateTime'] = date_df['Content'].str.extract(r'(\d{2}-\d{2}\s\d{2}:\d{2}:\d{2}\.\d{3})')
    date_df['DateTime'] = pd.to_datetime(date_df['DateTime'], format="%m-%d %H:%M:%S.%f")

    # Filter log entries within the specified time range
    mask = (date_df['DateTime'] >= from_date) & (date_df['DateTime'] <= to_date)
    filtered_df = date_df[mask]

    # Print the filtered log entries
    print("Filtered Log Entries:")
    #print(filtered_df)
    return filtered_df,date_df

# Example date and time
date = "02-21"
time = "00:38:41.336"
df,date_df = filterLogs(date, time,lines)
df.head(10)

In [ ]:
Enter the application name and check for fatal error in the given time.    
Enter the application and sub application name search for Fatal in the given time. 	
Enter application name and if log file not having any exception and Fatal and check PID within time period extract info based on PID and display.
Search through Module name.
If we don`t know the application name need to go with time and check for fatal or exception. If no fatal or exception then go with the module name.
START u0 --> keyword and search with timing and extract the information.




1. time Ui pass - df - rows - Fatal exception - generate report
2. after upload create a df and if the application name/subapplication/module and given time is found in any of the rows,get the exception info in report.
3. if no exception then get pid        ??????
4. if START u0  and time -are given then get that line last 
 

application and time 

com.sony.dtv.iswpowersavingsettings.input

iswpowersavingsettings = Package
input - subpackage

In [ ]:
import os
#root_folder = 'C:\Backup\pyc\Cisco\Base_iLA_Service'# Replace with the path to your root folder
#root_folder = 'C:\Backup\pyc\Cisco\remainingFiles\Base_iLA_Service\applicationback/'
# Recursive function to process the subfolders
def process_subfolders(folder_path):
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            file_path = os.path.join(root, file)
            if file_path.endswith(".zip"):
                with zipfile.ZipFile(file_path, 'r') as zip_ref:
                    zip_ref.extractall(folder_path)
        for directory in dirs:
            dir_path = os.path.join(root, directory)
            process_subfolders(dir_path) # Recursively process subfolders
# Start processing from the root folder
process_subfolders(root_folder)

In [ ]:
import os
import lzma
import csv
import zipfile

# Specify the folder where your XZ files are located
folder_path =  r"C:\Vinay\nokia_taks\PR725681\PR725681"
# Create a list to store the names of extracted files
extracted_files = []
# Function to extract XZ files recursively
def extract_xz_files(folder_path):
    for root, dirs, files in os.walk(folder_path):
        for filename in files:
            if filename.endswith(".xz"):
                file_path = os.path.join(root, filename)
                output_path = os.path.splitext(file_path)[0]  # Remove .xz extension
                # Open the XZ file and extract its contents
                with lzma.open(file_path, "rb") as xz_file:
                    with open(output_path, "wb") as output_file:
                        output_file.write(xz_file.read())
                # Add the extracted file name (without path) to the list
                extracted_files.append(os.path.basename(output_path))
            elif filename.endswith(".zip"):
                with zipfile.ZipFile(filename, 'r') as zip_ref:
                    zip_ref.extractall(output_path)
        for directory in dirs:
            dir_path = os.path.join(root, directory)
            extract_xz_files(dir_path) # Recursively process subfolders
extract_xz_files(folder_path)

In [ ]:
def process_subfolders(folder_path):
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            file_path = os.path.join(root, file)
            if file.endswith('.py'):
                os.remove(file_path) # Delete Python files
        for directory in dirs:
            dir_path = os.path.join(root, directory)
            process_subfolders(dir_path) # Recursively process subfolders
# Start processing from the root folder
process_subfolders(root_folder)

In [26]:
from datetime import datetime, timedelta
import pandas as pd

def filterLogs(input_time, lines):
    print("Filter Log Files...")

    # Convert the input time to a datetime object
    input_time = datetime.strptime(input_time, "%H:%M:%S.%f")

    # Calculate the time range (1 minute before and after the input time)
    from_time = input_time - timedelta(minutes=1)
    to_time = input_time + timedelta(minutes=1)
    print("from_time:", from_time)
    print("to_time:", to_time)

    # Create a DataFrame from the log content
    time_df = pd.DataFrame(lines, columns=['Content'])

    # Extract and convert timestamps to datetime objects
    time_df['Time'] = time_df['Content'].str.extract(r'(\d{2}:\d{2}:\d{2}\.\d{3})')
    time_df['Time'] = pd.to_datetime(time_df['Time'], format="%H:%M:%S.%f")

    # Filter log entries within the specified time range
    mask = (time_df['Time'] >= from_time) & (time_df['Time'] <= to_time)
    filtered_df = time_df[mask]

    # Print the filtered log entries
    print("Filtered Log Entries:")
    return filtered_df, time_df

# Example with input_time
input_time = "00:38:41.336"
#lines = []  # Replace with your log lines
df, time_df = filterLogs(input_time, lines)
print(df.head(10))


Filter Log Files...
from_time: 1900-01-01 00:37:41.336000
to_time: 1900-01-01 00:39:41.336000
Filtered Log Entries:
                                              Content                    Time
2   02-21 00:37:45.324  3030  3061 D Compatibility... 1900-01-01 00:37:45.324
3   02-21 00:37:47.324  3030  3061 D Compatibility... 1900-01-01 00:37:47.324
4   02-21 00:37:56.666  3030  3883 I ActivityTaskM... 1900-01-01 00:37:56.666
5   02-21 00:38:41.325  3030  3139 D Compatibility... 1900-01-01 00:38:41.325
6   02-21 00:38:41.332  3030  4974 D Compatibility... 1900-01-01 00:38:41.332
7   02-21 00:38:41.335  7378  7378 D AndroidRuntim... 1900-01-01 00:38:41.335
8   02-21 00:38:41.324  3030  3061 D Compatibility... 1900-01-01 00:38:41.324
9   02-21 00:38:41.324  3030  3517 I ActivityTaskM... 1900-01-01 00:38:41.324
10  02-21 00:39:41.324  3030  3061 D Compatibility... 1900-01-01 00:39:41.324
